# Remittance to the Philippines – Data Cleaning & Preprocessing

**Dataset Source:**  
https://www.kaggle.com/datasets/joshbuttler/remittance-to-the-philippines

**Purpose:**  
Prepare raw remittance data for exploratory analysis, statistical modeling, and forecasting by:
- Handling missing values
- Standardizing date formats
- Normalizing currency values (if applicable)
- Cleaning categorical fields

In [ ]:
import pandas as pd
import numpy as np

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

In [ ]:
# Path configuration
RAW_DATA_PATH = "../data/raw/remittance_philippines.csv"
PROCESSED_DATA_PATH = "../data/processed/remittance_cleaned.csv"

# Load dataset
df = pd.read_csv(RAW_DATA_PATH)

print("Dataset shape:", df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include="all").T

In [ ]:
missing_summary = (
    df.isnull()
      .sum()
      .to_frame("missing_count")
      .assign(missing_pct=lambda x: 100 * x["missing_count"] / len(df))
      .sort_values("missing_pct", ascending=False)
)

missing_summary

In [ ]:
# Separate column types
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns

# Numerical: median imputation
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Categorical: mode / Unknown
for col in cat_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "Unknown")

In [ ]:
# Separate column types
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns

# Numerical: median imputation
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Categorical: mode / Unknown
for col in cat_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "Unknown")

In [ ]:
# Identify date-like columns
date_cols = [col for col in df.columns if "date" in col.lower() or "year" in col.lower()]
date_cols

In [ ]:
# Example: standardizing a 'date' column
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [ ]:
# If only year is available
if "year" in df.columns:
    df["year"] = df["year"].astype(int)


In [ ]:
# Example placeholders
if "currency" in df.columns and "amount" in df.columns:

    # Example fixed FX rates (replace with real FX data if available)
    FX_RATES = {
        "USD": 1.0,
        "PHP": 1/56,
        "AED": 0.27
    }

    df["amount_usd"] = df.apply(
        lambda x: x["amount"] * FX_RATES.get(x["currency"], np.nan),
        axis=1
    )

In [ ]:
# Trim whitespace and standardize case
for col in cat_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.title()
    )

In [ ]:
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    return ((series < Q1 - 1.5 * IQR) | (series > Q3 + 1.5 * IQR))

outlier_summary = {
    col: detect_outliers_iqr(df[col]).sum()
    for col in num_cols
}

pd.DataFrame.from_dict(outlier_summary, orient="index", columns=["outlier_count"])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import os
os.makedirs("../data/processed", exist_ok=True)

df.to_csv(PROCESSED_DATA_PATH, index=False)

print("Cleaned dataset saved to:", PROCESSED_DATA_PATH)